<a href="https://colab.research.google.com/github/Mehrdad93/Business-ML/blob/master/NLP_biz_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Developed by Mehrdad Mokhtari
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.offline as plt2
import plotly.graph_objs as go
import gensim
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models.callbacks import CallbackAny2Vec
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.manifold import TSNE

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from itertools import cycle
from wordcloud import WordCloud, ImageColorGenerator

!pip install adjustText
from adjustText import adjust_text
import multiprocessing


In [ ]:
df = pd.read_csv('/content/drive/My Drive/WS_nlp/data_full.csv', engine='python', encoding='utf-8', error_bad_lines=False)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
l1 = len(df)
# removing the first year of dataframe
df.sort_values(by='Date')
df = df[pd.to_datetime(df['Date'], errors='coerce') >= pd.to_datetime('2013-01-01')]
l2 = len(df)
print("The percentage of dataframe that was removed in %", (l1-l2)*100/l1)
# df.head(5)

In [ ]:
# Remove threads which contain "Ask me anything" or "AMA" with diff. variations
df = df[~df["Title"].str.contains("Ask me anything", na=False)]
df = df[~df["Title"].str.contains("ask me anything", na=False)]
df = df[~df["Title"].str.contains("Ask Me Anything", na=False)]
df = df[~df["Title"].str.contains(" AMA ", na=False)]
df = df[~df["Title"].str.contains("[AMA]", na=False)]
df = df[~df["Title"].str.contains("(AMA)", na=False)]

l3 = len(df)
print("The percentage of dataframe that was removed in %", (l2-l3)*100/l2)

In [ ]:
# Remove all titles with NAN
title_data = df[df['Title'].notnull()]
# Grab all the titles and conversations 
reddit_titles = title_data['Title']
reddit_conv = title_data['Conversation']
# Create a list of strings, one for each title and conversation
titles_list = [title for title in reddit_titles]
conv_list = [title for title in reddit_conv]
# Collapse the list of strings into a single long string for processing
big_title_string = ' '.join(titles_list)
big_conv_string = ' '.join(map(str, conv_list))

# Tokenize the string into words: creating bag of words
tokens_titles = word_tokenize(big_title_string)
tokens_conv = word_tokenize(big_conv_string)

# Remove non-alphabetic tokens, such as punctuation
words_titles = [word.lower() for word in tokens_titles if word.isalpha()]
words_conv = [word.lower() for word in tokens_conv if word.isalpha()]

# Filter out stopwords
stop_words = stopwords.words('english')
# Some words are added to stopwords
newStopWords = ['one','though','anyway','anyways','like','even','thought','thing',
                'etc','cuz','dude','hey','hahah','haha','lol', 'mid', 'aa'] 
stop_words.extend(newStopWords)

words_titles = [word for word in words_titles if not word in stop_words]
words_conv = [word for word in words_conv if not word in stop_words]

# remove duplicate words:
seen = set()
words_conv_new = []
for item in words_conv:
    if item not in seen:
        seen.add(item)
        words_conv_new.append(item)

seen2 = set()
words_titles_new = []
for item2 in words_titles:
    if item2 not in seen2:
        seen2.add(item2)
        words_titles_new.append(item2)


In [ ]:
# Replier location and occupation
# 1
# create a dataset with non NAN values for replies
replier_data = df[df['Replier'].notnull()]
# adding two new columns 
replier_data['Location'], replier_data['Occupation'] = [np.nan, np.nan]
replier_conv = replier_data['Conversation']
replier_conv_list = [title for title in replier_conv]
print(replier_conv_list[1])

replier_tokens_conv = [0]*len(replier_conv_list)
for i in range(len(replier_conv_list)):
    # list in a list
    replier_tokens_conv[i] = word_tokenize(str(replier_conv_list[i]))

replier_words_conv = [0]*len(replier_conv_list)
for i in range(len(replier_conv_list)):
    replier_words_conv[i] = [word.lower() for word in replier_tokens_conv[i] if word.isalpha()]
    # print('-'*50)
    replier_words_conv[i] = [word for word in replier_words_conv[i] if not word in stop_words]


In [ ]:
# 2
# Occupation: simil_final_5
temp = [0]*len(replier_conv_list)
for i in range(len(replier_conv_list)):
    temp[i] = [word for word in simil_final_5 if word in replier_words_conv[i]]
for i in range(len(replier_conv_list)):
    if not temp[i]:
       replier_data['Occupation'].values[i] = 0
    else:
       replier_data['Occupation'].values[i] = 1

# US cities/states: simil_final_6 
temp = [0]*len(replier_conv_list)
for i in range(len(replier_conv_list)):
    temp[i] = [word for word in simil_final_6 if word in replier_words_conv[i]]
for i in range(len(replier_conv_list)):
    if not temp[i]:
       replier_data['Location'].values[i] = 0
    else:
       replier_data['Location'].values[i] = 1 

# adding two new variables to the main dataset just for replies
out = replier_data.to_csv('output.csv', index = None, header=True)   

In [ ]:
# 1 just conv of seekers
title_data = df[df['Title'].notnull()]
# Grab all the titles and conversations 
reddit_conv = title_data['Conversation']
# Create a list of strings, one for each title and conversation
conv_list = [title for title in reddit_conv]

title_data['Location'], title_data['Occupation'] = [np.nan, np.nan]
seeker_tokens_conv = [0]*len(conv_list)
for i in range(len(conv_list)):
    # list in a list
    seeker_tokens_conv[i] = word_tokenize(str(conv_list[i]))

seeker_words_conv = [0]*len(conv_list)
for i in range(len(conv_list)):
    seeker_words_conv[i] = [word.lower() for word in seeker_tokens_conv[i] if word.isalpha()]
    # print('-'*50)
    seeker_words_conv[i] = [word for word in seeker_words_conv[i] if not word in stop_words]


In [ ]:
# 2
# Occupation: simil_final_5
temp = [0]*len(conv_list)
for i in range(len(conv_list)):
    temp[i] = [word for word in simil_final_5 if word in seeker_words_conv[i]]
for i in range(len(conv_list)):
    if not temp[i]:
       title_data['Occupation'].values[i] = 0
    else:
       title_data['Occupation'].values[i] = 1

# US cities/states: simil_final_6 
temp = [0]*len(conv_list)
for i in range(len(conv_list)):
    temp[i] = [word for word in simil_final_6 if word in seeker_words_conv[i]]
for i in range(len(conv_list)):
    if not temp[i]:
       title_data['Location'].values[i] = 0
    else:
       title_data['Location'].values[i] = 1 

# adding two new variables to the main dataset just for seekers questions
out2 = title_data.to_csv('output2.csv', index = None, header=True)  
title_data["Location"].mean()


In [ ]:
# 1 conv of entire seekres and repliers-total
reddit_conv = df['Conversation']
# Create a list of strings, one for each title and conversation
conv_list = [title for title in reddit_conv]

df['Location'], df['Occupation'] = [np.nan, np.nan]
seeker_tokens_conv = [0]*len(conv_list)
for i in range(len(conv_list)):
    # list in a list
    seeker_tokens_conv[i] = word_tokenize(str(conv_list[i]))

seeker_words_conv = [0]*len(conv_list)
for i in range(len(conv_list)):
    seeker_words_conv[i] = [word.lower() for word in seeker_tokens_conv[i] if word.isalpha()]
    # print('-'*50)
    seeker_words_conv[i] = [word for word in seeker_words_conv[i] if not word in stop_words]


In [ ]:
# 2-total
# Occupation: simil_final_5
temp = [0]*len(conv_list)
for i in range(len(conv_list)):
    temp[i] = [word for word in simil_final_5 if word in seeker_words_conv[i]]
for i in range(len(conv_list)):
    if not temp[i]:
       df['Occupation'].values[i] = 0
    else:
       df['Occupation'].values[i] = 1

# US cities/states: simil_final_6 
temp = [0]*len(conv_list)
for i in range(len(conv_list)):
    temp[i] = [word for word in simil_final_6 if word in seeker_words_conv[i]]
for i in range(len(conv_list)):
    if not temp[i]:
       df['Location'].values[i] = 0
    else:
       df['Location'].values[i] = 1 

# Entire dataset
out3 = df.to_csv('output3.csv', index = None, header=True)  
df["Location"].mean()


In [ ]:
# NEW
# create a new list of words which excludes not original locations
# can be expanded!
excluded_loc_1 = ["order", "source", "export", "purchase"]
excluded_loc_2 = ["to", "from"]

for i in range(len(conv_list)):
  for word in seeker_words_conv[i]:
    if word in excluded_loc_1:
      if word in excluded_loc_2:
        df['Location'].values[i] = 0
        #print("changed")
        continue

out4 = df.to_csv('output4.csv', index = None, header=True) 
df["Location"].mean()


In [ ]:
_# Load Google's pre-trained Word2Vec model. ~100 billion words!
# It might take couple of minutes to be done
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/WS_nlp/GoogleNews-vectors-negative300.bin.gz', binary=True)

# model = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True) 
# model = gensim.models.KeyedVectors.load_word2vec_format('http://nlp.stanford.edu/data/wordvecs/glove.840B.300d.zip', binary=True, encoding='utf-8')

In [ ]:
# Filter the list of vectors to include only those that Word2Vec has a vector for
vector_list_titles = [model[word] for word in words_titles if word in model.vocab]
vector_list_conv = [model[word] for word in words_conv if word in model.vocab]

vector_list_conv_new = [model[word] for word in words_conv_new if word in model.vocab]
words_filtered_conv_new = [word for word in words_conv_new if word in model.vocab]

Vec = np.vstack(vector_list_conv_new) 
#Vec = np.vstack(vector_list_titles)

# Create a list of the words corresponding to these vectors
words_filtered_titles = [word for word in words_titles if word in model.vocab]
words_filtered_conv = [word for word in words_conv if word in model.vocab]

# Zip the words together with their vector representations
word_vec_zip_titles = zip(words_filtered_titles, vector_list_titles)
word_vec_zip_conv = zip(words_filtered_conv, vector_list_conv)

# Cast to a dict so we can turn it into a DataFrame
word_vec_dict_titles = dict(word_vec_zip_titles)
word_vec_dict_conv = dict(word_vec_zip_conv)
df_titles = pd.DataFrame.from_dict(word_vec_dict_titles, orient='index')
df_conv = pd.DataFrame.from_dict(word_vec_dict_conv, orient='index')
#df_conv.head(5)

In [ ]:
# 1 Similar words: country 
word_countries = ['canada', 'netherlands', 'denmark','australia','china']
simil_1 = model.wv.most_similar(positive=word_countries, negative=['viagra', 'macintosh', 'png', 'api'], topn=100) #opt
print('-'*30)
first_list = [x[0] for x in simil_1]
simil_final_2 = [word for word in first_list if word in words_conv]
simil_final_vec = [model[word] for word in first_list if word in words_conv]
# pd array 
Vec_2 = np.vstack(simil_final_vec) 
print(simil_final_2)
print(len(Vec_2))


In [ ]:
# 2 Similar words: season/month
word_season = ['january', 'February', 'march', 'summer', 'autumn', 'spring']
simil_2 = model.wv.most_similar(positive = word_season, topn = 150) #opt
print('-'*30)
first_list = [x[0] for x in simil_2]
simil_final_3 = [word for word in first_list if word in words_conv]
simil_final_vec = [model[word] for word in first_list if word in words_conv]
# pd array 
Vec_3 = np.vstack(simil_final_vec) 
print(simil_final_3)
print(len(Vec_3))


In [ ]:
# 3 Similar words: disrespectful
word_rude = ['crap', 'fuck', 'fucking', 'ass', 'idiot', 'shit', 'asshole']
simil_3 = model.wv.most_similar(positive = word_rude, topn = 200) #opt
print('-'*30)
first_list = [x[0] for x in simil_3]
simil_final_4 = [word for word in first_list if word in words_conv]
simil_final_vec = [model[word] for word in first_list if word in words_conv]
# pd array 
Vec_4 = np.vstack(simil_final_vec) 
print(simil_final_4)
print(len(Vec_4))


In [ ]:
# 4 Similar words: occupation
word_occupation = ['demolition', 'plumbing','barber','teacher','salesman']
simil_4 = model.wv.most_similar(positive = word_occupation, topn = 200) #opt
print('-'*30)
first_list = [x[0] for x in simil_4]
simil_final_5 = [word for word in first_list if word in words_conv]
simil_final_vec = [model[word] for word in first_list if word in words_conv]
# pd array 
Vec_5 = np.vstack(simil_final_vec) 
print(simil_final_5)
print(len(Vec_5))


In [ ]:
# 5 Similar words: city/state
word_city = ['houston','toronto','vancouver','nyc']
simil_5 = model.wv.most_similar(positive = word_city, topn = 100) #opt
print('-'*30)
first_list = [x[0] for x in simil_5]
simil_final_6 = [word for word in first_list if word in words_conv]
simil_final_vec = [model[word] for word in first_list if word in words_conv]
# pd array 
Vec_6 = np.vstack(simil_final_vec) 
print(simil_final_6)
print(len(Vec_6))


In [ ]:
# Dimensionality reduction: Initializing t-SNE to visualize locations and occupations in 2D
tsne0 = TSNE(n_components = 2, init = 'pca', random_state = 0, perplexity = 200)

tsne_df_conv = tsne0.fit_transform(Vec_5)
tsne_df_conv_1 = tsne0.fit_transform(Vec_2)

In [ ]:
# Just tsne_df_conv and df_conv plotted (country+occupation)
sns.set()
# Initialize figure
fig, ax = plt.subplots(figsize = (14, 10))
sns.scatterplot(tsne_df_conv[:, 0]+200, tsne_df_conv[:, 1]+200, alpha = 0.5)
sns.scatterplot(tsne_df_conv_1[:, 0], tsne_df_conv_1[:, 1], alpha = 0.5)

# Initialize list of texts
texts = []
texts_1 = []
words_to_plot = list(np.arange(0, len(Vec_5), 1))
words_to_plot_1 = list(np.arange(0, len(Vec_2), 1))

# Append words to list
for word in words_to_plot:
    texts.append(plt.text(tsne_df_conv[word, 0]+200, tsne_df_conv[word, 1]+200, simil_final_5[word], fontsize = 12))

for word1 in words_to_plot_1:
    texts_1.append(plt.text(tsne_df_conv_1[word1, 0], tsne_df_conv_1[word1, 1], simil_final_2[word1], fontsize = 12))

# Plot text using adjust_text (because overlapping text is hard to read)
adjust_text(texts, force_points = 0.4, force_text = 0.4, 
            expand_points = (2,1), expand_text = (1,2),
            arrowprops = dict(arrowstyle = "-", color = 'black', lw = 0.5))

adjust_text(texts_1, force_points = 0.4, force_text = 0.4, 
            expand_points = (2,1), expand_text = (1,2),
            arrowprops = dict(arrowstyle = "-", color = 'green', lw = 0.5))

plt.show()

In [ ]:
# 3D TSNE for tsne_df_conv and df_conv [new]
#Vec = np.vstack(vector_list_conv_new) 
def get_coordinates(Vec):
    tsne = TSNE(n_components=3, perplexity=100, init='random', random_state=0, verbose=1)
    # init='random', perplexity=100 ?? must try bigger preplexities as the dataset is big ~500
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(Vec)
    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    z_coords = Y[:, 2]
    return x_coords, y_coords, z_coords

# changed to Vec_new
x1, y1, z1 = get_coordinates(Vec_2)
x2, y2, z2 = get_coordinates(Vec_3)
x3, y3, z3 = get_coordinates(Vec_4)
x4, y4, z4 = get_coordinates(Vec_5)
x5, y5, z5 = get_coordinates(Vec_6)

plot1 = go.Scatter3d(x = x1,
                     y = y1, 
                     z = z1,
                     mode = 'markers+text',
                     text = simil_final_2, 
                     textposition = 'bottom center',
                     hoverinfo = 'text',
                     marker = dict(size=5, line=dict(color='rgba(1, 1, 1)', width=1), opacity=0.8))

plot2 = go.Scatter3d(x = x2+500,
                     y = y2,
                     z = z2+500,
                     mode = 'markers+text',
                     text = simil_final_3,
                     textposition = 'bottom center',
                     hoverinfo = 'text',
                     marker = dict(size=5, line=dict(color='rgb(30, 30, 30)', width=1), opacity=0.9))

plot3 = go.Scatter3d(x = x3,
                     y = y3-1000,
                     z = z3,
                     mode = 'markers+text',
                     text = simil_final_4,
                     textposition = 'bottom center',
                     hoverinfo = 'text',
                     marker = dict(size=5, line=dict(color='rgb(70, 70, 70)', width=1), opacity=0.9))

plot4 = go.Scatter3d(x = x4+1000,
                     y = y4,
                     z = z4+1000,
                     mode = 'markers+text',
                     text = simil_final_5,
                     textposition = 'bottom center',
                     hoverinfo = 'text',
                     marker = dict(size=5, line=dict(color='rgb(100, 100, 100)', width=1), opacity=0.9))

plot5 = go.Scatter3d(x = x5,
                     y = y5+1000, 
                     z = z5,
                     mode = 'markers+text',
                     text = simil_final_6, 
                     textposition = 'bottom center',
                     hoverinfo = 'text',
                     marker = dict(size=5, line=dict(color='rgba(150, 150, 150)', width=1), opacity=0.8))

data = [plot1, plot2, plot3, plot4, plot5]
layout = go.Layout(title='Clusters of various similar words in 3D space')
fig = go.Figure(data=data, layout=layout)
plt2.iplot(fig)


In [ ]:
# Clustring on conv (cluster the words using KMeans--> clustering algorithm look at differences between vectors/centers)
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters=num_clusters, init='k-means++', 
                               n_init=100, max_iter=500, tol=0.0001, verbose=0)
    idx = kmeans_clustering.fit_predict(word_vectors)
    return kmeans_clustering.cluster_centers_, idx

num_clusters = 20
centers, clusters = clustering_on_wordvecs(Vec, num_clusters)
centroid_map = dict(zip(words_filtered_conv_new, clusters))


In [ ]:
def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs)
    # Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers]
    closest_words_idxs = [x[1] for x in closest_points]
    # Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {}
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
    # A DataFrame is generated from the dictionary.
    df1 = pd.DataFrame(closest_words)
    return df1

top_words = get_top_words(words_filtered_conv_new, int(len(clusters)/500), centers, Vec) # 100< smallll


In [ ]:
# generating 20 clusters using kmeans clustering which are representing 
# similar words in each cluster
def display_cloud(cluster_num, cmap):
    wc = WordCloud(background_color="black", max_words=350, max_font_size=60, colormap=cmap);
    wordcloud = wc.generate(' '.join([word for word in top_words['Cluster #' + str(cluster_num)]]))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    plt.savefig('cluster_' + str(cluster_num), bbox_inches='tight')

# colors setting
cmaps = cycle(['flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
               'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg', 'hsv',
               'gist_rainbow', 'rainbow', 'jet', 'nipy_spectral', 'gist_ncar'])

for i in range(num_clusters):
    col = next(cmaps)
    display_cloud(i, col)

In [ ]:
# new: Document (title) to Vec
# Averaging Word Embeddings method
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in model.vocab]
    return np.mean(model[doc], axis=0)

# Our earlier preprocessing was done when we were dealing only with word vectors, here, we need each document to remain a document 
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] 
    return doc

# Function that will help us drop documents that have no word vectors in word2vec
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the w2v dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)

# Filter out documents
def filter_docs(corpus, texts, condition_on_doc):
    """
    Filter corpus and texts given the function condition_on_doc which takes a doc. The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)
    if texts is not None:
       texts = [text for (text, doc) in zip(texts, corpus) if condition_on_doc(doc)]

    corpus = [doc for doc in corpus if condition_on_doc(doc)]
    print("{} docs removed".format(number_of_docs - len(corpus)))
    return (corpus, texts)


In [ ]:
# Preprocess the corpus
# "titles_list" is already generated
corpus = [preprocess(title) for title in titles_list]

# Remove docs that don't include any words in W2V's vocab
corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: has_vector_representation(model, doc))

# Filter out any empty docs
corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: (len(doc) != 0))
x = []
for doc in corpus: # append the vector for each document
    x.append(document_vector(model, doc))
    
X = np.array(x) # list to array
print(len(X))

In [ ]:
# t-SNE, Round 2: Document Vectors
# Initialize t-SNE
tsne = TSNE(n_components = 2, init = 'pca', random_state = 0, perplexity = 100)   #best set of parameters
len_arr = 6000 #represent ~4-6 months of forum

for i in range(12):
    plt.close()
    h = len_arr*i
    tsne_df = tsne.fit_transform(X[h:h+len_arr])

    fig, ax = plt.subplots(figsize = (20, 15))
    sns.scatterplot(tsne_df[:, 0], tsne_df[:, 1], alpha = 0.3)
    texts = []
    titles_to_plot = list(np.arange(0, len_arr, 400)) 
    # Append words to list
    for title in titles_to_plot:
        texts.append(plt.text(tsne_df[title, 0], tsne_df[title, 1], titles_list[title+h], fontsize = 13)) 
    
    # Plot text using adjust_text
    adjust_text(texts, force_points = 0.5, force_text = 0.5, 
                expand_points = (2,1), expand_text = (1,2),
                arrowprops = dict(arrowstyle = "-", color = 'red', lw = 0.6))
    
    plt.savefig("img{:02}.png".format(i), dpi=300)
    plt.show()
    

In [ ]:
# # Cluster the titles using KMeans--> clustering algorithm look at differences between vectors/centers)
# def clustering_on_wordvecs(word_vectors, num_clusters):
#     # Initalize a k-means object and use it to extract centroids
#     kmeans_clustering = KMeans(n_clusters=num_clusters, init='k-means++', 
#                                n_init=100, max_iter=500, tol=0.0001, verbose=0)
#     idx = kmeans_clustering.fit_predict(word_vectors)
    
#     return kmeans_clustering.cluster_centers_, idx

# num_clusters = 5
# # X represents the new vector for docs (titles)
# # titles_list represents each title in a list
# centers, clusters = clustering_on_wordvecs(X, num_clusters) #on half of the X
# centroid_map = dict(zip(titles_list, clusters))
# print('Done!', num_clusters) 

In [ ]:
# def get_top_titles(index2word, k, centers, wordvecs):
#     tree = KDTree(wordvecs)
#     # Closest points for each Cluster center is used to query the closest 20 points to it.
#     closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers]
#     closest_words_idxs = [x[1] for x in closest_points]
#     # Word Index is queried for each position in the above array, and added to a Dictionary.
#     closest_words = {}
#     for i in range(0, len(closest_words_idxs)):
#         closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
#     # A DataFrame is generated from the dictionary.
#     df1 = pd.DataFrame(closest_words)
#     return df1

# top_titles = get_top_titles(titles_list, len(clusters), centers, X) 

# # Separate each top_titles into a specific category
# online_list = top_titles[0]
# len(online_list)

# idea_list = top_titles[1]
# len(idea_list)

# opinion_list = top_titles[2]
# len(opinion_list)

# sale_list = top_titles[3]
# len(sale_list)

# marketing_list = top_titles[4]
# len(marketing_list)

In [ ]:
df1 = title_data[title_data['Title'].str.contains("e-commerce") | title_data['Title'].str.contains("E-commerce") | title_data['Title'].str.contains("digital") | title_data['Title'].str.contains("social media") | title_data['Title'].str.contains("platform") |title_data['Title'].str.contains("blog") | title_data['Title'].str.contains("website") | title_data['Title'].str.contains('amazon')  | title_data['Title'].str.contains('Amazon') | title_data['Title'].str.contains('online') | title_data['Title'].str.contains('site')]
idea_titles = df1['Title']
online_list = [title for title in idea_titles]
len(online_list)

df1 = title_data[title_data['Title'].str.contains("idea") | title_data['Title'].str.contains("plan")]
idea_titles = df1['Title']
idea_list = [title for title in idea_titles]
len(idea_list)

df1 = title_data[title_data['Title'].str.contains("feedbacks") | title_data['Title'].str.contains("feedback") | title_data['Title'].str.contains("opinion") | title_data['Title'].str.contains("advice") | title_data['Title'].str.contains('suggestion') | title_data['Title'].str.contains("recommendation") | title_data['Title'].str.contains("opinions") | title_data['Title'].str.contains("advices") | title_data['Title'].str.contains('suggestions') | title_data['Title'].str.contains("recommendations") ]
idea_titles = df1['Title']
opinion_list = [title for title in idea_titles]
len(opinion_list)

df1 = title_data[title_data['Title'].str.contains("sale") | title_data['Title'].str.contains("sales")]
idea_titles = df1['Title']
sale_list = [title for title in idea_titles]
len(sale_list)

df1 = title_data[title_data['Title'].str.contains("marketing")]
idea_titles = df1['Title']
marketing_list = [title for title in idea_titles]
len(marketing_list)


In [ ]:
# Creating dictionary with mentioned categories
foo_0 = {'E-commerce': online_list,
         'Business Idea': idea_list,
         'Opinions/Feedbacks': opinion_list,
         'Sales': sale_list,
         'Marketing': marketing_list}

foo_1 = pd.DataFrame.from_dict(foo_0, orient='index')
foo_1 = foo_1.transpose()
foo_1.to_csv("list_titles.csv", index=False)

len_rest = len(X) - int((len(online_list)+len(idea_list)+len(opinion_list)+len(sale_list)+len(marketing_list))*1.0)
labels = ['E-commerce', 'Business Idea', 'Opinions/Feedbacks', 'Sales', 'Marketing', 'The rest']
values = [int(len(online_list)*1.0), int(len(idea_list)*1.0), int(len(opinion_list)*1.0), int(len(sale_list)*1.0), int(len(marketing_list)*1.0), len_rest]
# Pie chart with Plotly
fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0.1, 0, 0, 0, 0])])
fig.update_layout(title="The percentage of 'Business Idea' titles versus the other hot topics: ")
fig.show()

In [ ]:
# Adding “BizIdea” which is binary with 1 being in the “Business Ideas” cluster and 0 being not
df['BizIdea'] = np.nan
list_titles = list(df["Title"])

for i in range(len(list_titles)):
    if list_titles[i] in idea_list:
       df['BizIdea'].values[i] = 1
    else:
       df['BizIdea'].values[i] = 0

out5 = df.to_csv('output5.csv', index = None, header=True)  

In [ ]:
# related them together

In [ ]:
df_new = pd.read_csv('/content/drive/My Drive/WS_nlp/MyData.csv', engine='python', encoding='utf-8', error_bad_lines=False)
for i in df_new.columns.values:
  T[i] = df_new.columns.values[i]
# print(T)

In [ ]:
# do not run (a comment chunk)
# topic 1
'idea market book step course agency youtube area lead capital firm channel hack run guidance scratch reality minute validation content passion flippa size train growth date creation worker brainstorm inspire'
# topic 2
'experience time pay thing tool subscription box resource kickstarter opinion guide skill investment value franchise break production check spend art record sticker fba factory builder magazine hold advertisement beer bag'
# topic 3
'product interest amazon deal life purchase drop datum space world office mail photo struggle seller family member scale concept nothing location right expand expense raise dropshipper morning operate produce ppc'
# topic 4
'start online business design price story development offer manufacture restaurant web delivery technology moment mastermind season holiday broker position export woocommerce ticket accountant impact proposal potential slack compare foot prep'
# topic 5
'sale feedback customer partner project today tax strategy form change license detail term register copy stay motivation distributor asium government stand rep agent premium automate exist apartment prelaunch cut motivate'
# topic 6
'service list email account instagram review card case study credit example issue follower twitter failure relate setup photography info vendor party paper database limit couple mobile forum enter text connection'
# topic 7
'website software base cost reddit investor custom host target feedback feel xpost event inventory rstartup file prototype shirt quality room management bar insight commerce iphone criticism progress bring document secure'
# topic 8
'app ecommerce store blog china shopify venture equity article online point result march gift write noob blogger dream mistake curiou movie split innovation giveaway usa squarespace seek wonder shoot pain'
# topic 9
'person video industry hire llc food content car cofounder survey show outsource structure everyone partnership trade support join flip piece message percentage july reseller power task con acquisition click response'
# topic 10
'startup facebook niche place kind patent country solution talk competition application manager read image bill invoice man estate visitor generation begin decision supplement return career writer flyer expert require boost'
# topic 11
'page land game marketplace network import contact competitor track freelance tech journey insurance end taxe mentor board difference series challenge focu somebody copyright andor answer summer energy display submit coach'
# topic 12
'advice research college group student conversion user validate audience practice machine mine theme fulfillment gain australium reason town hmy invention send volume join round contest night scam path cold provide'
# topic 13
'name brand launch plan seo friend interview promote domain link bank house growth loan marketer internet retailer ceo trend europe grant switch choose auto liability accelerator presence description trainer background'
# topic 14
'site create recommendation critique employee traffic package feature method finance freelancer discussion niche input computer repair field sort oversea woman hair pokemon gold programmer experiment dog session burn poster avoid'
# topic 15
'tip learn web developer side lesson buy manufacturer request alibaba increase dollar accomplishment project vium dev university buyer productivity june pursue trial distribution hear bulk focus parent deck security equipment'
# topic 16
'sell suggestion shop income source print search tshirt model regard move state ebook online ebay demand schedule watch corporation rent keyword amount custom style enterprise leverage notice stop merchandise fire'
# topic 17
'money medium job management founder community hour quit recommend budget margin advantage stage future contractor fashion discount promotion portfolio weekend commission chance brick simple decide tomorrow mortar kid subscriber transfer'
# topic 18
'affiliate build payment profit program domain anybody type worth drive supplier hunt adword turn fail amazon biz law coffee traffic monetization pick technique direction truck stripe visit claim artist legality'
# topic 19
'ship platform order campaign problem home system approach pitch manage consult goal stuff figure property update apparel organization sport hand item thread letter youve printer poll conference ability face fix'
# topic 20
'post owner advertise google opportunity revenue grow word trademark facebook fee blog subreddit adsense betum newsletter reach view merchant sample news rank etsy resell content exchange stick trump topic'
# topic 21
'share client option team podcast rate phone information contract stock number canada risk sign comment process relationship lease stream door agreement picture attempt crm estimate calculate foundation cannabis miss'
# topic 22
'fund clothe school item line charge monetize label generate travel engineer meet knowledge influencer assistant trouble benefit paypal situation quote californium health level center consumer matter rule adult webinar'
# topic 23
'test logo designer city b2b consultant sub cash code template handle music care release report download beginner mind mvp fun lawyer teach hate edit tea waste supply material construction convert'


In [ ]:
# Topic modeling
# X has been created in the previous project (document vector)
topic = X
topic_vec = topic[0:int(0.8*len(X))]
train_vecs = []
for i in range(len(topic_vec)):
    # frequency of 0.8 chosen
    topic_vec = [topic_vec[i] for i in range(23)]
    train_vecs.append(topic_vec)


In [ ]:
# Training a Supervised Classifier
X = np.array(train_vecs)
y = np.array(df_new.target)

kf = KFold(5, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # Logistic Regression SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=23,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='binary'))


In [ ]:
# Applying the Model on Unseen Data

def get_bigram(df):
    df['text'] = strip_newline(df.text)
    words = list(sent_to_words(df.text))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    return bigram
  
bigram_test = get_bigram(df_new)

test_corpus = [train_id2word.doc2bow(text) for text in bigram_test]

topic_vec = topic[int(0.8*len(X)):]
test_vecs = []

for i in range(len(topic_vec)):
    topic_vec = [topic_vec[i] for i in range(23)]
    topic_vec.extend([df_new.iloc[i].real_counts]) 
    topic_vec.extend([len(df_new.iloc[i].text)]) 
    test_vecs.append(topic_vec)


# create a dataset with non NAN values for replies
replier_data = df[df['Replier'].notnull()]
replier_data['Topic Number'] = np.nan
for topic in T:
  temp = [0]*len(replier_conv_list)
  n += 1
  for i in range(len(replier_conv_list)):
      temp[i] = [word for word in topic if word in replier_words_conv[i]]
      replier_data['Topic Number'].values[i] = n

out = replier_data.to_csv('topic_reply.csv', index = None, header = True)  

In [ ]:
data['Topic Loading'] = np.nan
data['Topic Loading'] = data['Topic Loading'].astype(object)

# Create a 23 by 2 array
a = np.ndarray(shape=(23,2))
a[:,0] = range(1,24)

# Vectors of replies: "topic" (3 chunks above)
# Run this loop on Cedar or other HPCs (with 60G RAM-CPU on Cedar took around 23 hours to be completed)
for i in range(len(data)):
    if data['Topic Number'].notnull().values[i]:
        aa = a
        temp_ = data['Topic Number'].values[i]
        for vector in topic:
          for j in vector:
            norm[j] = float(vector[j])/sum(vector) #23 classes
            aa[j,1] = norm[j]
        if int(max(aa[:,1])) != int(aa[temp_-1,1]):
            print("Oops!")
            break
        b = np.ndarray.tolist(aa)
        # treated as an object [b]
        data.loc[i, 'Topic Loading'] = [b]
        # check if the code is running
        if i%10000 == 0:
          print(i)

out_2 = replier_data.to_csv('Topic_Loading.csv', index=None, header=True)  